In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

In [29]:
data1 = pd.read_csv("phone_user_review_file_1.csv", encoding = 'latin1')
data2 = pd.read_csv("phone_user_review_file_2.csv", encoding = 'latin1')
data3 = pd.read_csv("phone_user_review_file_3.csv", encoding = 'latin1')
data4 = pd.read_csv("phone_user_review_file_4.csv", encoding = 'latin1')
data5 = pd.read_csv("phone_user_review_file_5.csv", encoding = 'latin1')
data6 = pd.read_csv("phone_user_review_file_6.csv", encoding = 'latin1')

In [30]:
frames = [data1, data2, data3, data4, data5, data6]
data = pd.concat(frames)

In [31]:
data.shape

(1415133, 11)

In [32]:
data.columns

Index(['phone_url', 'date', 'lang', 'country', 'source', 'domain', 'score',
       'score_max', 'extract', 'author', 'product'],
      dtype='object')

In [33]:
data.isnull().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
dtype: int64

In [34]:
data.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [35]:
data.drop(columns = ['date', 'lang', 'country', 'source', 'domain', 'score_max', 'extract', 'product'], inplace = True)

In [36]:
data.head()

,phone_url,score,author
0,/cellphones/samsung-galaxy-s8/,10.0,CarolAnn35
1,/cellphones/samsung-galaxy-s8/,10.0,james0923
2,/cellphones/samsung-galaxy-s8/,6.0,R. Craig
3,/cellphones/samsung-galaxy-s8/,9.2,Buster2020
4,/cellphones/samsung-galaxy-s8/,4.0,S Ate Mine


In [37]:
data.dtypes

phone_url     object
score        float64
author        object
dtype: object

In [38]:
# imp=SimpleImputer(missing_values=np.nan, strategy = 'mean')
# idata=pd.DataFrame(imp.fit_transform(data)) # imputed data
# idata.columns=data.columns
imp_value = data['score'].median()
data.fillna(imp_value, inplace = True)

In [39]:
data.drop_duplicates(keep = 'first', inplace = True)

In [40]:
data = data.sample(n = 1000000, random_state = 612)

In [41]:
data = data.round(decimals = 0)

In [42]:
data.shape

(1000000, 3)

In [43]:
data.groupby('phone_url').size().agg(['idxmax','max'])

idxmax    /cellphones/samsung-galaxy-s5/
max                                11356
dtype: object

Thus the most rated phone is samsung galaxy s5. 

In [44]:
data.groupby('author')['author'].count().sort_values(ascending = False)

author
9.2                8328
Amazon Customer    4788
Cliente Amazon     1679
Anonymous          1496
Amazon Kunde       1168
                   ... 
adrianamur            1
adrianacl             1
adriana tuÃ±as        1
adriana torres        1
Rebeki                1
Name: author, Length: 699568, dtype: int64

The author 9.2 has the highest number of ratings, 8328. 

In [45]:
data_new = data.copy()
data_new['user_rating_count'] =  data.groupby('author')['author'].transform('count')
data_new['phone_rating_count'] = data.groupby('phone_url')['phone_url'].transform('count')

In [46]:
data_new = data_new.loc[((data_new['user_rating_count'] >= 50) & (data_new['phone_rating_count'] >= 50))]

In [47]:
data_new.shape

(84708, 5)

In [48]:
data_new.drop(columns = ['user_rating_count'], inplace = True)

In [49]:
data_new.groupby('phone_url')['score', 'phone_rating_count'].mean().sort_values(by = ['score', 'phone_rating_count'], ascending = False).head(10)

<ipython-input-49-145c2943643e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_new.groupby('phone_url')['score', 'phone_rating_count'].mean().sort_values(by = ['score', 'phone_rating_count'], ascending = False).head(10)


,score,phone_rating_count
phone_url,,
/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,10.0,160
/cellphones/samsung-sgh-s275/,10.0,77
/cellphones/sony-ericsson-t230/,10.0,76
/cellphones/motorola-i860/,10.0,70
/cellphones/motorola-m3788/,10.0,60
/cellphones/karbonn-titanium-x/,10.0,55
/cellphones/siemens-a60/,10.0,55
/cellphones/vodafone-smart-mini/,10.0,53
/cellphones/prestigio-multiphone-4300-duo/,10.0,52


as we can see, top 5 mobile phones are samsung-galaxy-s-iii-slim-sm-g3812, samsung-sgh-s275, sony-ericsson-t230, motorola-i860 and motorola-m3788.   

In [50]:
from surprise import Dataset,Reader

reader = Reader(rating_scale=(1, 10))
data_s = Dataset.load_from_df(data_new[['phone_url', 'author', 'score']], reader)

In [51]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data_s, test_size=.25,random_state=123)

In [52]:
trainset.all_ratings()

<generator object Trainset.all_ratings at 0x00000219DF64FA50>

In [81]:
from surprise import SVD, KNNWithMeans
from surprise import accuracy
from surprise.model_selection import cross_validate

### SVD based Recommendation system

In [92]:
svd_model = SVD(n_factors=5,biased=False)
svd_model.fit(trainset)

In [93]:
test_pred = svd_model.test(testset)
test_pred

[Prediction(uid='/cellphones/samsung-galaxy-s-ii/', iid='Rahul', r_ui=4.0, est=7.417373785906065, details={'was_impossible': False}),
 Prediction(uid='/cellphones/sony-ericsson-c702-148206/', iid=9.2, r_ui=6.0, est=7.453601880103333, details={'was_impossible': False}),
 Prediction(uid='/cellphones/apple-iphone-5c/', iid='Michael', r_ui=8.0, est=10, details={'was_impossible': False}),
 Prediction(uid='/cellphones/nokia-206/', iid='Amazon Customer', r_ui=10.0, est=6.468882302998951, details={'was_impossible': False}),
 Prediction(uid='/cellphones/nokia-1650/', iid='Andrea', r_ui=4.0, est=9.079335033853946, details={'was_impossible': False}),
 Prediction(uid='/cellphones/nokia-n70/', iid='dal07', r_ui=9.0, est=9.186205200386164, details={'was_impossible': False}),
 Prediction(uid='/cellphones/microsoft-lumia-640-xl/', iid='Manuel', r_ui=10.0, est=8.386384623370116, details={'was_impossible': False}),
 Prediction(uid='/cellphones/lg-g2-mini/', iid='Andrea', r_ui=8.0, est=8.455441918200442,

In [94]:
accuracy.rmse(test_pred)

RMSE: 2.8219


2.821892022229013

In [95]:
cross_validate(algo_i, data_s, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.9869  2.9994  2.9760  2.9768  2.9772  2.9833  0.0090  
MAE (testset)     2.3619  2.3808  2.3655  2.3604  2.3644  2.3666  0.0073  
Fit time          2.61    2.95    3.06    2.98    3.19    2.96    0.19    
Test time         6.37    7.28    6.70    6.64    6.74    6.75    0.30    


{'test_rmse': array([2.98686098, 2.99937267, 2.97603786, 2.97682863, 2.97715657]),
 'test_mae': array([2.36191073, 2.38080578, 2.36553726, 2.36043123, 2.36442569]),
 'fit_time': (2.6062068939208984,
  2.945688247680664,
  3.064965009689331,
  2.9808900356292725,
  3.1862409114837646),
 'test_time': (6.365238904953003,
  7.278331279754639,
  6.704622507095337,
  6.636125802993774,
  6.744640111923218)}

### KNNWithMeans based recommendation system : item based

In [96]:
algo_i = KNNWithMeans(k=10, sim_options={ 'user_based': False})

algo_i.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [97]:
test_pred=algo_i.test(testset)
test_pred

[Prediction(uid='/cellphones/samsung-galaxy-s-ii/', iid='Rahul', r_ui=4.0, est=6.082495266584038, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='/cellphones/sony-ericsson-c702-148206/', iid=9.2, r_ui=6.0, est=6.933680377734835, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='/cellphones/apple-iphone-5c/', iid='Michael', r_ui=8.0, est=6.333617235247916, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='/cellphones/nokia-206/', iid='Amazon Customer', r_ui=10.0, est=7.020567622218436, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='/cellphones/nokia-1650/', iid='Andrea', r_ui=4.0, est=8.862922141988522, details={'actual_k': 8, 'was_impossible': False}),
 Prediction(uid='/cellphones/nokia-n70/', iid='dal07', r_ui=9.0, est=9.0, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='/cellphones/microsoft-lumia-640-xl/', iid='Manuel', r_ui=10.0, est=7.6616996324491655, details={'actual_k': 10, 'wa

In [98]:
print(accuracy.rmse(test_pred))

RMSE: 2.9344
2.9344095987319685


In [99]:
cross_validate(algo_i, data_s, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.9224  2.9332  2.9477  2.9416  2.9278  2.9345  0.0091  
MAE (testset)     2.2836  2.2907  2.3061  2.3052  2.2925  2.2956  0.0087  
Fit time          0.35    0.39    0.35    0.34    0.36    0.36    0.02    
Test time         1.48    1.46    1.41    1.53    1.47    1.47    0.04    


{'test_rmse': array([2.92241636, 2.93315404, 2.94767661, 2.94156851, 2.92776298]),
 'test_mae': array([2.28362602, 2.29069242, 2.30607381, 2.305157  , 2.29246646]),
 'fit_time': (0.3462502956390381,
  0.38899660110473633,
  0.349743127822876,
  0.3429906368255615,
  0.3564760684967041),
 'test_time': (1.4805941581726074,
  1.4556984901428223,
  1.4122071266174316,
  1.5278751850128174,
  1.4705233573913574)}

### KNNWithMeans based recommendation system : user based

In [100]:
algo_i = KNNWithMeans(k=10, sim_options={ 'user_based': True})

algo_i.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [101]:
test_pred=algo_i.test(testset)
test_pred

[Prediction(uid='/cellphones/samsung-galaxy-s-ii/', iid='Rahul', r_ui=4.0, est=7.060097388734861, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='/cellphones/sony-ericsson-c702-148206/', iid=9.2, r_ui=6.0, est=7.081182913456982, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='/cellphones/apple-iphone-5c/', iid='Michael', r_ui=8.0, est=7.901793590176961, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='/cellphones/nokia-206/', iid='Amazon Customer', r_ui=10.0, est=6.980908463517159, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='/cellphones/nokia-1650/', iid='Andrea', r_ui=4.0, est=6.4500922071195355, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='/cellphones/nokia-n70/', iid='dal07', r_ui=9.0, est=9.0258082746912, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='/cellphones/microsoft-lumia-640-xl/', iid='Manuel', r_ui=10.0, est=7.822122542256054, details={'actua

In [102]:
print(accuracy.rmse(test_pred))

RMSE: 2.9990
2.999002487807852


In [103]:
cross_validate(algo_i, data_s, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.9700  2.9783  2.9951  3.0073  2.9791  2.9860  0.0134  
MAE (testset)     2.3593  2.3579  2.3793  2.3832  2.3609  2.3681  0.0108  
Fit time          3.21    3.80    2.96    3.32    4.21    3.50    0.45    
Test time         8.98    7.64    6.75    9.86    8.01    8.25    1.08    


{'test_rmse': array([2.96997378, 2.97829448, 2.99509648, 3.00732567, 2.97914455]),
 'test_mae': array([2.35928265, 2.35792738, 2.37925652, 2.3831825 , 2.36085661]),
 'fit_time': (3.2133378982543945,
  3.7966501712799072,
  2.9564743041992188,
  3.3166942596435547,
  4.205559730529785),
 'test_time': (8.976345539093018,
  7.643560171127319,
  6.74910306930542,
  9.856934785842896,
  8.006017208099365)}

we can see that without cross-validation, SVD based recommendation system has the least RMSE, but with cross-validation, KNNWithMeans item based recommendation system has the least mean RMSE. Thus, KNNWithMeans item based recommendation system is the best system. 

10: When we want to recommend phones to a user without knowing the user's identity, we would use popularity based recommendation systems. 

11: When we want to use the user's identity in recommending him phones, we would use collaborative filtering based recommendation systems. 

12: We can add price range for the recommendation systems, so that users looking to buy phones in a particular pre-determined price range find it easy to do so. 